# ToDo: 
- criticality of intersections. should all of them be nodes?
- try building network without OSM data [done]
- add coordinates to nodes to plot the network? [done]
- try building network with OSM data (first,requires I add width to OSM data)
- create network of all roads, with weights [done]
- investigate algorithm for filling up links randomly with weight threshold

# Road Graph network of Milan

## Data preprocessing

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import networkx as nx

### Import and initial cleaning

In [ ]:
vehicle_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/DBT_2020/SHAPE/AC_VEI_AC_VEI_SUP_SR.shp"
gdf = gpd.read_file(vehicle_path)

In [ ]:
gdf.drop(['AC_VEI_FON', 'AC_VEI_LIV', 'AC_VEI_SED', 'CLASSREF'],axis = 1, inplace = True)
gdf.rename(columns={'SUBREGID':'ID', 'NOME': 'NAME', 'AC_VEI_ZON': 'TYPE'}, inplace = True)

pattern1 = ('01','02')
 # portions of road (e.g not intersections or parking lots) start with 01 in TYPE
 # intersections, squares, and roundabouts start with 02 in TYPE
gdf = gdf[~gdf['NAME'].str.contains('TANGENZIALE', regex = False)] #removing tangenziali
gdf = gdf[gdf.TYPE.str.startswith(pattern1)]

We use the same crs as OSM:

In [ ]:
OSM_crs = 3857
gdf.to_crs(epsg=OSM_crs, inplace = True)

### Some useful functions:


In [ ]:
#divides gdf into intersections and roads

def ints_and_roads(gdf):

    pattern2 = ('01')
    pattern3 = ('02') 
    pattern4 = ('0102')
    roads = gdf[(gdf.TYPE.str.startswith(pattern2)) & ~ (gdf.TYPE.str.startswith(pattern4))]
    ints = gdf[(gdf.TYPE.str.startswith(pattern3)) | (gdf.TYPE.str.startswith(pattern4))]
    return ints, roads

In [ ]:
#function creates geodataframe with all streets of gdf within distance dist (in meters) of street.
#street is a geodataframe, dist is a positive number, and gdf is the geodataframe dataset.

def within_dist(street, dist, gdf):

    temp = street.copy()
    temp.geometry = temp.geometry.buffer(dist)
    temp = temp.filter(['geometry']) #so sjoin doesn't give suffixes and i don't have to rename later
    gdf_distanced = gdf.sjoin(temp, how='inner', predicate='intersects')
    gdf_distanced = gdf_distanced.dropna()
    gdf_distanced = gdf_distanced[~gdf_distanced.index.duplicated(keep='first')] #removes streets that are in more than one polygon's buffer
    gdf_distanced = gdf_distanced.iloc[:,:-1] #drops index_R column
    return gdf_distanced

In [ ]:
#we define a variation of the within_dist function. This one keeps duplicate entries because they are useful for creating the graph later on.

def within_dist_dupes(street, dist, gdf):

    temp = street.copy()
    temp.geometry = temp.geometry.buffer(dist)
    temp = temp.filter(['geometry']) #so sjoin doesn't give suffixes and i don't have to rename later
    gdf_distanced = gdf.sjoin(temp, how='inner', predicate='intersects')
    gdf_distanced = gdf_distanced.dropna()
    return gdf_distanced

### Average width calculation: 
Area is length times width for rectangles.  
Perimeter is 2(length) + 2(width)  
$A = lw$  
$P = 2l+2w$  
brings us to solve for width as   

$P = 2\frac{A}{w}+2w$ 
so  
$w^2 -\frac{P}{2}w+A = 0$

In [ ]:
#calculates width of all entries in gdf, and adds them to a width column. Assumes rectangular equivalent shape for polygons

def calc_widths(gdf):
    gdf['temp'] = 1 # create column of ones
    gdf['SemiPeri'] = -gdf.length/2 # i need it negative for the equation
    gdf['Area'] = gdf.area
    def calculate_roots(row):
        coefficients = row[['temp', 'SemiPeri', 'Area']].values
        roots = np.roots(coefficients).real
        return roots

    #gdf['roots'] = gdf.apply(calculate_roots, axis=1)
    gdf['roots'] = gdf[['temp', 'SemiPeri', 'Area']].apply(calculate_roots, axis=1)
    gdf[['root1', 'root2']] = pd.DataFrame(gdf['roots'].tolist(), index=gdf.index)
    gdf['width'] = gdf['root2']
    gdf = gdf.drop([ 'Area', 'temp', 'SemiPeri', 'roots', 'root2'], axis = 1)
    return gdf

In [ ]:
gdf = calc_widths(gdf)

NB with this method root2 is always the width because all segments of road are longer than they are wide, and root2 is always the smaller of the two roots, by construction of the method. at most there can be entries where root1=root2, which would mean that they both equal the width.

## Useful subdivisions
We can add info into our dataset about neighborhoods, municipal zones, and even just custom subdivisions of streets within a certain
radius of another.  
These are all useful for visualization of the dataset, and for testing algorithms on subsets smaller than the entire dataset.

In [ ]:
from geopandas.tools import sjoin

### Municipal info

In [ ]:
municipal_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/DBT_2020_new/SHAPE/Municipi.shp"
gdf_M = gpd.read_file(municipal_path)
gdf_M = gdf_M.to_crs(epsg = OSM_crs)

gdf = gdf.sjoin(gdf_M, how = 'inner',predicate = 'intersects') # requires gpd > 0.9
#gdf_zone = gpd.sjoin(gdf_roads_piaz, gdf2, how = 'inner', op = 'intersects') #is equivalent, with older syntax
gdf = gdf.drop(['AREA', 'PERIMETRO', 'index_right'], axis =1)

### Neighborhood info

In [ ]:
neighborhood_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/Quartieri milano_real/NIL_WM.shp"
gdf_N = gpd.read_file(neighborhood_path)
gdf_N = gdf_N.to_crs(epsg = OSM_crs)
gdf_N = gdf_N.drop(['Valido_dal', 'Fonte', 'Shape_Leng', 'Shape_Area', 'OBJECTID', 'Valido_al'] ,axis=1)


gdf = gdf.sjoin(gdf_N, how = 'inner',predicate = 'intersects')
gdf = gdf.drop(['index_right'], axis = 1)


Finally, let's split our main dataframe into two: one with only roads, and one with only intersections.

In [ ]:
ints, roads = ints_and_roads(gdf)

### Example: The Stadera Neighborhood of Milan

In [ ]:
#Now isolate an example neighborhood
tot_Stadera = gdf[gdf['NIL'] == 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA']
ints_Stadera, roads_Stadera = ints_and_roads(tot_Stadera)
#and a single street in that neighborhood
tot_Volv = tot_Stadera[tot_Stadera['NAME'] == 'VIA VOLVINIO']
ints_Volv, roads_Volv = ints_and_roads(tot_Volv)

In [ ]:
fig, ax = plt.subplots(2,1, figsize = (8,8))

ax[0].set_title(f"The Stadera Neighborhood of Milan")
tot_Stadera.plot(ax = ax[0], alpha = 0.5, column = 'TYPE')
#plt.legend(title='Type', labels=tot_Stadera['TYPE'].unique())
cx.add_basemap(ax[0], crs=gdf_N.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
ax[1].set_title("and a single street from it")
tot_Volv.plot(ax = ax[1], alpha = 0.5, column = 'NAME')
cx.add_basemap(ax[1], crs=gdf_N.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite

plt.show()

### Example 2: Creating custom ranges by distance

We use within_dist to create a Geodataframe with all roads within a given distance from the road given as input

In [ ]:
#start with an example road (you can even use more than one road)
tot_Volv = gdf[gdf['NAME'] == 'VIA VOLVINIO']
ints_Volv, roads_Volv = ints_and_roads(tot_Volv)

M = 100 #distance in meters
temp = within_dist(tot_Volv, M, gdf)


In [ ]:
fig, ax = plt.subplots(1,2, figsize = (8,8))

temp.plot(ax = ax[0], alpha = 0.5, column = 'TYPE')
plt.legend(title='Type', labels=temp['TYPE'].unique())
cx.add_basemap(ax[0], crs=gdf.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
plt.title(f"roads within {M} meters from Via Volvinio")
temp.plot(ax = ax[1], alpha = 0.5, column = 'NAME')
cx.add_basemap(ax[1], crs=gdf.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite

plt.show()

## Final preprocessing steps: fixing adjacent roads.

Here is one of the criticalities: for some reason, some intersections are connected by chains of "runway" polygons with no intersections between them.  
For example, here we see two nodes of the network that should be connected, that don't count as connected in the graph because they are separated by more than one polygon. you can tell from the plot on the right that the polygons are in fact separate.

In [ ]:
meda = gdf[gdf.NAME == 'VIA GIUSEPPE MEDA']
meda_int = ints[ints.NAME == 'VIA GIUSEPPE MEDA']

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (4,4))
k = within_dist(meda.loc[2277:2280], 10, gdf)
k.plot(column = 'TYPE', ax = ax[0])
k.plot(column = 'ID', ax = ax[1])
plt.show()

### Solution implementation:  


#### 1) Unary union 

The easiest, most automatic method to get adjacent geometries into one is using unary union. It automatically gives me what i need. The problem is that i lose all info which is not geometry. I can get everything from TYPE to NIL back easily but, i'd lose NAME and ID. 
I create a new dataset with all the roads (obtained through iteration...) and then add the dataset with the intersections to it.

Dissolve method is conceptually more complicated, but more efficient and elegant. See below

In [ ]:
q = gdf.copy()
q.to_crs(epsg = OSM_crs, inplace = True)
pattern1 = ('01','02')
q = q[~q['NAME'].str.contains('TANGENZIALE', regex = False)] #removing tangenziali
q = q[q['TYPE'].str.startswith(pattern1)]
q.reset_index(inplace = True, drop = True)
intq, nonq = ints_and_roads(q)

one for intersections and one for non intersections. we need to recreate our q dataset with all our roads made adjacent. it takes some steps.

In [ ]:
qq = gpd.geoseries.GeoSeries([geom for geom in nonq.unary_union.geoms])


In [ ]:
qqq = gpd.geoseries.GeoSeries([geom for geom in intq.unary_union.geoms])
#do i really need to do this too? hmmm

NB in doing this we're losing info on the names of the streets. I'm not a huge fan of this fact.

In [ ]:
print(len(qqq), len(intq))

In [ ]:
print([len(nonq), len(qq)])

In [ ]:
new_intq = gpd.GeoDataFrame(geometry = qqq, crs = OSM_crs)
new_intq['TYPE'] = '02'
new_intq.set_geometry('geometry');

In [ ]:
new_nonq = gpd.GeoDataFrame(geometry = qq, crs = OSM_crs)
new_nonq['TYPE'] = '01'
new_nonq.set_geometry('geometry');

In [ ]:
intq = calc_widths(new_intq)

nonq = calc_widths(new_nonq)

In [ ]:
neighborhood_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/Quartieri milano_real/NIL_WM.shp"

nonq = nonq.to_crs(epsg = OSM_crs)
nonq = nonq.sjoin(gdf_N, how = 'inner', predicate = 'intersects')
nonq = nonq.drop(['index_right'], axis = 1)
intq = intq.sjoin(gdf_N, how = 'inner', predicate = 'intersects')
intq = intq.drop(['index_right'], axis = 1)

In [ ]:
nonq_S = nonq[nonq.NIL == 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA']

recreating single dataset:

In [ ]:
q = pd.concat([intq,nonq])

#### 2) Dissolve

Starting from entire dataset, divide into intersections, and non intersections (roads).  
From there, roads with adjacent roads will be the complementary set of within_dist(int, 1, non_int).
This is because within_dist(...) gives all non_ints (roads) that are beside intersections.  
Now we have roads not adjacent to any intersection; we're missing the roads that these roads are adjacent to.  
we use within_dist_dupes(roads, 1, non_int) to get all our chains of roads, grouped together by an 'index_right' column.  
we use dissolve(by = 'index_right') to make each chain of roads into a single one.  
We take the complement of the result of within_dist_dupes(...), and concat it with our dissolved gdf.  
This is now the entire road dataset. Concat this with the intersections, and we have now treated our dataset properly and can proceed with making our graph.  

NB I tested this on a few roads manually (namely VIA GIUSEPPE MEDA, VIA ASCANIO SFORZA etc.) and everything seemed fine. I imagine there may be some problems with longer chains of roads, but generally I'm quite confident that the amount of critical cases is negligible.

In [ ]:
w = gdf.copy()

In [ ]:
#takes gdf, dissolves adjacent roads into single roads, and replaces them into the original gdf

def road_dissolver(gdf):
    ints, roads = ints_and_roads(gdf)
    there = within_dist(ints, 1, roads) #all roads that have adjacent intersections
    not_there = roads[~roads.index.isin(there.index)] #all roads that don't
    adj = within_dist_dupes(not_there, 1, roads)
    not_dissolved = roads[~roads.index.isin(adj.index)] # i build complement before dissolving because dissolving doesn't preserve all indices
    dissolved = adj.dissolve(by = 'index_right')
    #now concat back to retrieve entire dataset
    new_roads = pd.concat([dissolved, not_dissolved])
    new_gdf = pd.concat([new_roads, ints])
    new_gdf.reset_index(inplace = True, drop = True)
    return new_gdf
    

In [ ]:
w = road_dissolver(w)

In [ ]:
w = calc_widths(w)

# The Network
Make intersections the nodes, and make roads the edges. road width are the weights.  
NB for now we will consider all streets in the manner which is most convenient, i.e as two-way streets, unless otherwise specified.
Due to how the dataset is built, sometimes there are two consecutive roads without intersections. We must slightly simplify the dataset so that this isn't an issue.


remember:
- intersections as nodes.
- the actual roads as edges.
- their width and name will be weights/edge attributes.


To find the edges of the network, we take an intersection and use the within_dist function with a distance = 1 to find
all roads that are immediately adjacent to the intersection. 
These are the "stubs" of our graph, i.e lines that connect to a node and nothing else.  
When we do this for all intersections, the edges of the network will simply be the common stubs between pairs of nodes.  
Our road network will be a MultiGraph, because some intersections may be connected by two or more different roads.

This function is only called on dataframes with indexes from 0 to N.  
When we sjoin an intersection with its surrounding streets, our resulting dataframe will tell us, in each row, the index of the intersection
that the given road is adjacent to. this information will be contained in "index_right", which is the new name given to what was the intersection index in the original dataframe.  

This means each row of our sjoined dataframe represents a stub of the node "index_right".  


We divide into N dataframes based on index_right, to have a dataframe of the adjacent edges to one road (i.e the stubs).   
there is a connection between dataframe_i and dataframe_j for each common stub between the two.  

Finally, the function that performs these operations and finds the connections is the following:

In [ ]:
def make_edges(gdf_tot):

    #prep
    #takes dataset with roads and intersections, creates edgelist of nodes with weights of edges
    pattern1, pattern2, pattern3 = '01', '02', '0102'
    no_ints = gdf_tot[(gdf_tot.TYPE.str.startswith(pattern1)) & ~ (gdf_tot.TYPE.str.startswith(pattern3))]
    ints = gdf_tot[(gdf_tot.TYPE.str.startswith(pattern2)) | (gdf_tot.TYPE.str.startswith(pattern3))]
    #we need indices from 0 --> reset
    ints.reset_index(inplace = True, drop = True)
    no_ints.reset_index(inplace = True, drop = True)


    #body
    stubs = within_dist_dupes(ints, 1, no_ints) #all stubs, i.e all roads connected to all nodes
    grouped = stubs.groupby('index_right') #one dataframe for each node
    edges = {} # will contain intersections of each node 
    edge_list = pd.DataFrame(columns = ['from','to','weight'])
    for node, group in grouped:
        stubs = stubs[stubs['index_right'] != node] #removing "self" from gdf that we will merge onto, to avoid self connections. also removes redundancies  
        edges[node] = pd.merge(group,stubs, on = 'ID', how = 'inner')
        edge_list_temp = pd.DataFrame({'from': edges[node].index_right_x, 'to': edges[node].index_right_y, 'weight': edges[node].width_x})
        edge_list = pd.concat([edge_list if not edge_list.empty 
                               else None,edge_list_temp])
    
    
    #exceptions    
    #adds self-edges to nodes that don't appear in to or from
    conc = pd.concat([edge_list['from'], edge_list['to']])
    all = set(range(0, len(ints))) #all possible nodes
    there = set(conc.unique()) #the nodes we actually have
    not_there = sorted(list(all-there)) #missing nodes (irregardless of why they're missing for the moment)
    df_self = pd.DataFrame({'from': not_there, 'to': not_there, 'weight': [1] * len(not_there)})
    df_self
    edge_list = pd.concat([edge_list, df_self])
    return edge_list
    


## Demostration of a problem:
When working with an UNPROCESSED gdf:
After extensive testing, it seems clear that making the network works quite well on a small scale, but:
at a certain point you will encounter a series of consecutive roads without an intersection.  
Unfortunately, without handling these exceptions, the total network ends up being divided into many giant components.  
This doesn't make sense, since a road network should be connected.  
That is why we preprocess our gdf with unary union or with dissolve.

In [ ]:
M = 1400
temp = within_dist(tot_Volv, M, gdf)


In [ ]:
edges = make_edges(temp)
G = nx.from_pandas_edgelist(edges, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())

Gcc = sorted(nx.connected_components(G), key = len, reverse = True)
print(len(Gcc[0]), len(Gcc[1]),len(Gcc[2]))

intt, nont = ints_and_roads(temp)

This already shows that the second and third largest components are much larger than they should be

In [ ]:
cent = intt.centroid
coordinates = np.column_stack((cent.geometry.x, cent.geometry.y))
positions = dict(zip(sorted(G.nodes), coordinates))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))
#temp.plot(marker=".", column= "TYPE", ax=ax, alpha = 0.5) to add underlying polygons

ax.set_title("Partial road graph, with clusters highlighted")
ax.axis("off")
colorlist = [ 'r', 'g', 'b', 'y', 'orange']
#create subgraphs, get nodelist and edgelist. pass to the two functions. possible cmap of edges by width
for i in range(0, len(Gcc)):
    nx.draw_networkx_nodes(G, positions, nodelist = list(Gcc[i]), 
                           node_color = colorlist[i%5], ax=ax, 
                           node_size=10)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, positions, edge_color = weights, 
                       edge_cmap = plt.cm.inferno, 
                       edge_vmin = min(weights), edge_vmax = max(weights),
                       ax = ax)    
#labels = nx.draw_networkx_labels(G, pos=positions, font_size = 6)
cx.add_basemap(ax, source=cx.providers.CartoDB.DarkMatterNoLabels)

plt.show()

In [ ]:
#plt.colormaps()

We can see that there are large componenents of the network that never connect, even though there are two nodes that look like there should be a connection between them.  
That is why we preprocess with unary_union or Dissolve

### Unary union Network

The following section requires the "Unary Union" section to have been executed in order to work.

In [ ]:
q['ID'] = q.index #need to have ID column for "make_edges" to work
edge_list = make_edges(q)
G = nx.from_pandas_edgelist(edge_list, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())

In [ ]:
Gcc = sorted(nx.connected_components(G), key = len, reverse = True)
#print(len(Gcc[0]), len(Gcc[1]),len(Gcc[2]))
intq, nonq = ints_and_roads(q)
print(len(Gcc[0]), len(intq)) #to see how large the giant component is comparted to total nodes

In [ ]:
#add positional information to nodes:

cent = intq.centroid
coordinates = np.column_stack((cent.geometry.x, cent.geometry.y))
positions = dict(zip(sorted(G.nodes), coordinates))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))
#temp.plot(marker=".", column= "TYPE", ax=ax, alpha = 0.5)

ax.set_title("Partial road graph, with clusters highlighted")
ax.axis("off")
colorlist = [ 'r', 'g', 'b', 'y', 'orange']
#create subgraphs, get nodelist and edgelist. pass to the two functions. possible cmap of edges by width
for i in range(0, len(Gcc)):
    nx.draw_networkx_nodes(G, positions, nodelist = list(Gcc[i]), 
                           node_color = colorlist[i%5], ax=ax, 
                           node_size=1.5)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, positions, edge_color = weights, 
                       edge_cmap = plt.cm.inferno, 
                       edge_vmin = min(weights), edge_vmax = max(weights),
                       ax = ax)    
#labels = nx.draw_networkx_labels(G, pos=positions, font_size = 6)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

plt.show()

### Dissolve Network

The following section requires the "Dissolve" section to have been executed in order to work.

In [ ]:
edge_list = make_edges(all)
G = nx.from_pandas_edgelist(edge_list, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())

In [ ]:
Gcc = sorted(nx.connected_components(G), key = len, reverse = True)
#print(len(Gcc[0]), len(Gcc[1]),len(Gcc[2]))
ints, roads = ints_and_roads(all)
print(len(Gcc[0]), len(ints)) #to see how large the giant component is comparted to total nodes

In [ ]:
cent = ints.centroid
coordinates = np.column_stack((cent.geometry.x, cent.geometry.y))
positions = dict(zip(sorted(G.nodes), coordinates))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))
#temp.plot(marker=".", column= "TYPE", ax=ax, alpha = 0.5)

ax.set_title("Partial road graph, with clusters highlighted")
ax.axis("off")
colorlist = [ 'r', 'g', 'b', 'y', 'orange']
#create subgraphs, get nodelist and edgelist. pass to the two functions. possible cmap of edges by width
for i in range(0, len(Gcc)):
    nx.draw_networkx_nodes(G, positions, nodelist = list(Gcc[i]), 
                           node_color = colorlist[i%5], ax=ax, 
                           node_size=1, alpha = 0.3)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, positions, edge_color = weights, 
                       edge_cmap = plt.cm.inferno, 
                       edge_vmin = min(weights), edge_vmax = max(weights),
                       ax = ax)   
#labels = nx.draw_networkx_labels(G, pos=positions, font_size = 6)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

plt.show()

Way more self edges (approx 500), although most of them near the boundaries of the graph. need to understand why.

## Percolation trials


My goal for now is to find the percolation threshold based on width. In other words, does my network have a giant component? how many wide roads can i remove before i don't have one anymore? and vice versa

In [ ]:
Gc = sorted(nx.connected_components(G), key = len, reverse = True)

In [ ]:
S = len(Gc[0])/len(G) #percentage of network occupied by largest connected component.
S

NB S is good for preliminary analysis because it treats all nodes and edges equally. we may want to add some classification of importance of which nodes we remove later on.

In [ ]:
edgesss=sorted(G.edges(data=True), key=lambda edge: edge[2].get('weight', 1)) #sorts edges by width

create range of min to max width and do various removals, then plot S vs width

In [ ]:
r = np.linspace(edge_list.weight.min(), edge_list.weight.max()/2, 30)
S_list = {}
S2_list= {} #second largest component list
for i in r:
    edge_list_t = edge_list[edge_list.weight > i]
    Gt = nx.from_pandas_edgelist(edge_list_t, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())
    Gcc = sorted(nx.connected_components(Gt), key=len, reverse=True)
    S_list[i] = len(Gcc[0])/len(G)
    if len(Gcc) > 1:
        S2_list[i] = len(Gcc[1])/len(G)
    else: S2_list[i] = 0

breakdown clearly happens between 12 and 13m street removal.

In [ ]:
#fig, ax = subplots(1,1, figsize(8,8))
keys = [float(key) for key in S_list.keys()]

# Extract values
S_values = list(S_list.values())
S2_values = list(S2_list.values())

fig,ax = plt.subplots(1,1,figsize = (6,6))
ax2 = ax.twinx()
# Plot keys vs values
ax.plot(keys, S_values, marker='o')
ax2.plot(keys, S2_values, color = 'red')
ax.set_xlabel('min width of streets kept (m)')
ax.set_ylabel('fractional size of giant component')
ax2.set_ylabel('second largest component')
plt.title('Percolation test')
plt.grid(True)
plt.show()

### now removing highest to lowest

In [ ]:
r = np.linspace(edge_list.weight.max()/2, edge_list.weight.min()+0.1,  20)
S_list_rev = {}
S2_list_rev = {}
for i in r:
    edge_list_t = edge_list[edge_list.weight < i]
    Gt = nx.from_pandas_edgelist(edge_list_t, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())
    Gcc = sorted(nx.connected_components(Gt), key=len, reverse=True)
    S_list_rev[i] = len(Gcc[0])/len(G)
    S2_list_rev[i] = len(Gcc[1])/len(G)

In [ ]:
#fig, ax = subplots(1,1, figsize(8,8))
keys = [float(key) for key in S_list_rev.keys()]

# Extract values
S_rev_values = list(S_list_rev.values())
S2_rev_values = list(S2_list_rev.values())

fig,ax = plt.subplots(1,1,figsize = (6,6))
ax2 = ax.twinx()
# Plot keys vs values
ax.plot(keys, S_rev_values, marker='o')
ax2.plot(keys, S2_rev_values, color = 'red')
ax.set_xlabel('max width of streets kept (m)')
ax.set_ylabel('fractional size of giant component')
ax2.set_ylabel('second largest component')
plt.title('Percolation test')
plt.grid(True)
plt.show()

# Network with OSMnx package (incomplete)

to be completed

In [ ]:
import osmnx as ox

In [ ]:
G = ox.graph_from_place("Rome, Italy", network_type = 'drive')

In [ ]:
M = ox.graph_from_place("Milan, Lombardy, Italy", network_type = 'drive')

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(M)
gdf_nodes.head()

In [ ]:
# what sized area does our network cover in square meters?
M_proj = ox.project_graph(M)
nodes_proj = ox.graph_to_gdfs(M_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_m

In [ ]:
# show some basic stats about the network
ox.basic_stats(M_proj, area=graph_area_m, clean_int_tol=15)

In [ ]:
# convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(M))
nx.set_edge_attributes(M, edge_centrality, "edge_centrality")

# color edges in original graph with closeness centralities from line graph
ec = ox.plot.get_edge_colors_by_attr(M, "edge_centrality", cmap="inferno")
fig, ax = ox.plot_graph(M, edge_color=ec, edge_linewidth=2, node_size=0)

#NB takes about 13 minutes to execute for milan

Just like at the beginning, our problem with having an osmnx network is that there is no width attribute on these roads. How can we do what we need to without width? we can't calculate it because the road network is not made up of polygons, but of lines.

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8,8))
gdf_nodes.plot(markersize = 0.1, ax = ax, color = 'red')
gdf_edges.plot(linewidth = 0.5, ax = ax, alpha = 0.5)

plt.show()

Let's test the method of calculating width with a Voronoi tessellation.
first let's take two adjacent nodes, calculate their distance and then create a voronoi tessellation between them by using geoplot.voronoi

In [ ]:
#get a node and its neighbors

M.nodes

In [ ]:
n = 10371529
adj = M.adj[n]

In [ ]:
adj_nodes, adj_edges = ox.graph_to_gdfs(adj)

In [ ]:
nodes_proj